# SymPy amplitude model

In [ ]:
# To run in in Google Colab, uncomment the following:

# !pip install expertsystem graphviz

In [ ]:
import expertsystem as es

result = es.reaction.generate(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)(980)", "f(0)(1500)"],
    allowed_interaction_types=["strong", "EM"],
)

In [ ]:
import graphviz

graphs = result.collapse_graphs()
dot = es.io.convert_to_dot(graphs)
graphviz.Source(dot)

In [ ]:
model = es.amplitude.generate_sympy(result)
expression = model.expression
expression.full_expression

## Navigate through each of the components

In [ ]:
expression.top

In [ ]:
down_to_intensities = expression.top.subs(expression.intensities)
down_to_intensities

In [ ]:
down_to_amplitudes = down_to_intensities.subs(expression.amplitudes)
down_to_amplitudes

In [ ]:
down_to_dynamics = down_to_amplitudes.subs(expression.dynamics)
down_to_dynamics

In [ ]:
initial_values_substituted = down_to_dynamics.subs(
    {k: v.value for k, v in model.parameters.items()}
)
initial_values_substituted

In [ ]:
evaluated_wigner_d = initial_values_substituted.doit()
evaluated_wigner_d